# New Parsing Dev

In [1]:
from cayley_dickson_base import CayleyDicksonBase
from cayley_dickson_integers import Zi
from cayley_dickson_rationals import Qi
import generic_utils as utils

import re

## Test Quaternion String Parsing

In [2]:
from unittest import TestCase, TextTestRunner, defaultTestLoader
from cayley_dickson_integers import Zi
from random import seed

class TestZi(TestCase):

    def setUp(self) -> None:
        pass

    def test_parse_quaternion_string(self):
        self.assertEqual(Zi.parse_quaternion_string('1+2i+3j+4k'), [1, 2, 3, 4])
        self.assertEqual(Zi.parse_quaternion_string('-1+3i-3j+7k'), [-1, 3, -3, 7])
        self.assertEqual(Zi.parse_quaternion_string('-1-4i-9j-2k'), [-1, -4, -9, -2])
        self.assertEqual(Zi.parse_quaternion_string('17-16i-15j-14k'), [17, -16, -15, -14])
        self.assertEqual(Zi.parse_quaternion_string('7+2i'), [7, 2])
        self.assertEqual(Zi.parse_quaternion_string('7+2j'), [7, 0, 2, 0])
        self.assertEqual(Zi.parse_quaternion_string('2i-6k'), [0, 2, 0, -6])
        self.assertEqual(Zi.parse_quaternion_string('1-5j+2k'), [1, 0, -5, 2])
        self.assertEqual(Zi.parse_quaternion_string('1-5j'), [1, 0, -5, 0])
        self.assertEqual(Zi.parse_quaternion_string('3+4i-9k'), [3, 4, 0, -9])
        self.assertEqual(Zi.parse_quaternion_string('42i+j-k'), [0, 42, 1, -1])
        self.assertEqual(Zi.parse_quaternion_string('6-2i+j-3k'), [6, -2, 1, -3])
        self.assertEqual(Zi.parse_quaternion_string('1+i+j+k'), [1, 1, 1, 1])
        self.assertEqual(Zi.parse_quaternion_string('-1-i-j-k'), [-1, -1, -1, -1])
        self.assertEqual(Zi.parse_quaternion_string('16k-20j+2i-7'), [-7, 2, -20, 16])
        self.assertEqual(Zi.parse_quaternion_string('i+4k-3j+2'), [2, 1, -3, 4])
        self.assertEqual(Zi.parse_quaternion_string('5k-2i+9+3j'), [9, -2, 3, 5])
        self.assertEqual(Zi.parse_quaternion_string('5k-2j+3'), [3, 0, -2, 5])
        self.assertEqual(Zi.parse_quaternion_string('1.75-1.75i-1.75j-1.75k'), [1.75, -1.75, -1.75, -1.75])
        self.assertEqual(Zi.parse_quaternion_string('2.0j-3k+0.47i-13'), [-13, 0.47, 2.0, -3])
        self.assertEqual(Zi.parse_quaternion_string('(5.6-3i)'), [5.6, -3])
        self.assertEqual(Zi.parse_quaternion_string('k-7.6i'), [0, -7.6, 0, 1])
        self.assertEqual(Zi.parse_quaternion_string('0'), [0, 0])
        self.assertEqual(Zi.parse_quaternion_string('0j+0k'), [0, 0])
        self.assertEqual(Zi.parse_quaternion_string('-0j'), [0, 0])
        self.assertEqual(Zi.parse_quaternion_string('1-0k'), [1, 0])
        self.assertEqual(Zi.parse_quaternion_string('1+2i+3j+4K'), [1, 2, 3, 4])
        self.assertEqual(Zi.parse_quaternion_string('7.1E-2 +4.3k+i'), [0.071, 1, 0, 4.3])
        self.assertEqual(Zi.parse_quaternion_string('3 - 2E-3i-4j'), [3, -0.002, -4, 0])

In [3]:
TextTestRunner(verbosity=2).run(defaultTestLoader.loadTestsFromTestCase(TestZi))

test_parse_quaternion_string (__main__.TestZi.test_parse_quaternion_string) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.007s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## Parsing Experiments

In [14]:
qstrings = ['1+2i+3j+4k',
            '4k-3j-2i+1']

pat1 = r'[-+]?((\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?)?[ijk]'
# pat2 = r'[-+]?((\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?)[^ijk]'

for qstr in qstrings:
    for match in re.finditer(pat1, qstr):
        beg = match.start()
        end = match.end()
        trm = match.group(0)
        print(beg, end, trm)
    print('-----')

1 4 +2i
4 7 +3j
7 10 +4k
-----
0 2 4k
2 5 -3j
5 8 -2i
-----


In [62]:
qstrings = ['1+2i+3j+4k',
            '4k-3j-2i+1',
            '2i - 4 + k',
            'k',
            'i+j']

pat1 = r'[-+]?((\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?)?[ijk]'

for qs in qstrings:
    qstr = qs.strip().replace(' ', '')
    print(f"{qstr = }\n")
    terms = []
    real = None
    for match in re.finditer(pat1, qstr):
        beg = match.start()
        end = match.end()
        trm = match.group(0)
        print(beg, end, trm)
        terms.append(trm)
        # print(f"'{qstr[:beg] + trm + qstr[end:]}'")
    print(f"{terms = }")
    for tm in terms:
        qstr = qstr.replace(tm, '')
    if qstr == '':
        real = '0'
    else:
        real = qstr
    print(f"real = {repr(real)}")
    print('\n---------\n')

qstr = '1+2i+3j+4k'

1 4 +2i
4 7 +3j
7 10 +4k
terms = ['+2i', '+3j', '+4k']
real = '1'

---------

qstr = '4k-3j-2i+1'

0 2 4k
2 5 -3j
5 8 -2i
terms = ['4k', '-3j', '-2i']
real = '+1'

---------

qstr = '2i-4+k'

0 2 2i
4 6 +k
terms = ['2i', '+k']
real = '-4'

---------

qstr = 'k'

0 1 k
terms = ['k']
real = '0'

---------

qstr = 'i+j'

0 1 i
1 3 +j
terms = ['i', '+j']
real = '0'

---------



In [41]:
qstr
foo = qstr[:]

In [42]:
id(qstr)

4399919920

In [45]:
id(str(qstr))

4399919920

In [ ]:
text = '1+i+j+k'
pattern = r'[-+][lL]|[lL][ijk]|[ijk]'
char_to_splice = "1"
splice_position = 0 # Splice after '+' or '-'

match = re.search(pattern, text)

if match:
    start_index = match.start()
    end_index = match.end()
    matched_string = match.group(0)

    # Splice the character within the matched string
    modified_matched_string = (
        matched_string[:splice_position] +
        char_to_splice +
        matched_string[splice_position:]
    )

    # Reconstruct the full string
    new_text = (
        text[:start_index] +
        modified_matched_string +
        text[end_index:]
    )
    print(f"Original string: {text}")
    print(f"Modified string: {new_text}")
else:
    print("Pattern not found.")

In [ ]:
qstr = '1+i+j+k'
pat = r'[-+][lL]|[lL][ijk]|[ijk]'
# char_to_splice = "1"
# splice_position = 0 # Splice after '+' or '-'

In [ ]:
def add_implied_coefficients(text):

    pattern = r'[-+][lL]|[lL][ijk]|[ijk]'
    char_to_splice = "1"
    splice_position = 0 # Splice after '+' or '-'

    match = re.search(pattern, text)

    if match:
        start_index = match.start()
        end_index = match.end()
        matched_string = match.group(0)

        # Splice the character within the matched string
        modified_matched_string = (
            matched_string[:splice_position] +
            char_to_splice +
            matched_string[splice_position:]
        )

        # Reconstruct the full string
        new_text = (
            text[:start_index] +
            modified_matched_string +
            text[end_index:]
        )

        return new_text, True
    else:
        return text, False

In [ ]:
qstr = '1+i+j+k'
foo = add_implied_coefficients(qstr)
foo

In [ ]:
add_implied_coefficients(foo[0])

In [ ]:
mat = re.search(pat, qstr)
mat

In [ ]:
mat.start()

In [ ]:
mat.end()

In [ ]:
qstr[mat.start()]

In [ ]:
qstr[mat.end()]

In [ ]:
qstr1 = '1+i+j+k+L+Li+Lj+Lk'
qstr2 = '1+i+j+k+l+li+lj+lk'

In [ ]:
pat = r'[-+][ijklL]|[lL]i|[lL]j|[lL]k'

In [ ]:
qstr = qstr2

indices = []
for mat in re.finditer(pat, qstr):
    indices.append(mat.start())
    print(mat.start(), mat.end(), mat.group())
indices

In [ ]:
for idx in reversed(indices):
    qstr = qstr[:idx+1] + '1' + qstr[idx+1:]

In [ ]:
qstr

In [ ]:
def add_implied_coefficients(st):
    pat = r'[-+][ijklL]|[lL]i|[lL]j|[lL]k'
    indices = []
    # Find locations of implied coefficients
    for match in re.finditer(pat, st):
        indices.append(match.start())
    # Splice in the implied coefficients
    for idx in reversed(indices):
        st = st[:idx+1] + '1' + st[idx+1:]
    return st

In [ ]:
add_implied_coefficients(qstr1)

In [ ]:
add_implied_coefficients(qstr2)

In [ ]:
qstr3 = '1+2i-j-1k+L-Li-4Lj+Lk'

In [ ]:
print(qstr3)
print(add_implied_coefficients(qstr3))

In [ ]:
qstr4 = str(Zi.random_octonion())
qstr4

In [ ]:
add_implied_coefficients(qstr4)